In [1]:
import pandas as pd
from math import sin, cos, sqrt, atan2, radians
import time
import multiprocessing
import os
from multiprocessing import Pool
import multiprocessing.pool
import bs4 as bs
import urllib.request
import numpy as np

In [2]:
import fitz
import glob
import pandas as pd
import re
from operator import itemgetter
from itertools import groupby

In [3]:
broker_pdf_list = glob.glob('C:/Users/i26184/Desktop/Broker/Broker_PDF/*.pdf')

In [4]:
max([int(re.findall(r'\d+',x)[0]) for x in broker_pdf_list])

26184

In [5]:
broker_pdf_list

['C:/Users/i26184/Desktop/Broker/Broker_PDF\\individual_2498722.pdf',
 'C:/Users/i26184/Desktop/Broker/Broker_PDF\\individual_31169.pdf',
 'C:/Users/i26184/Desktop/Broker/Broker_PDF\\individual_816404.pdf',
 'C:/Users/i26184/Desktop/Broker/Broker_PDF\\individual_816436.pdf',
 'C:/Users/i26184/Desktop/Broker/Broker_PDF\\individual_816439.pdf',
 'C:/Users/i26184/Desktop/Broker/Broker_PDF\\individual_816452.pdf',
 'C:/Users/i26184/Desktop/Broker/Broker_PDF\\individual_816461.pdf',
 'C:/Users/i26184/Desktop/Broker/Broker_PDF\\individual_816467.pdf',
 'C:/Users/i26184/Desktop/Broker/Broker_PDF\\individual_816472.pdf',
 'C:/Users/i26184/Desktop/Broker/Broker_PDF\\individual_816473.pdf',
 'C:/Users/i26184/Desktop/Broker/Broker_PDF\\individual_816494.pdf',
 'C:/Users/i26184/Desktop/Broker/Broker_PDF\\individual_816500.pdf',
 'C:/Users/i26184/Desktop/Broker/Broker_PDF\\individual_816501.pdf',
 'C:/Users/i26184/Desktop/Broker/Broker_PDF\\individual_816506.pdf',
 'C:/Users/i26184/Desktop/Broker/B

In [6]:
len(broker_pdf_list)

107

In [7]:
broker_pdf_list = [x.replace('\\','/') for x in broker_pdf_list]

In [8]:
def get_broker_name_CRD(fitz_doc):
    page1 = fitz_doc.loadPage(0)
    page1_textblock = page1.getTextBlocks()
    page1_textblock = [x[4] for x in page1_textblock]
    broker_name_candiate = page1_textblock[0].split('\n')
    
    if len(broker_name_candiate)==1:
        broker_name = page1_textblock[1]
    else:
        broker_name = broker_name_candiate[1]
        
    section_start_index = [i for i in range(len(page1_textblock)) if page1_textblock[i]=='Section Title']
    section_end_index = [i for i in range(len(page1_textblock)) if 'CRD# ' in page1_textblock[i]]
    
    if len(section_end_index) == 0:
        CRD_number = 'NO CRD Number'
    else:
        CRD_number = page1_textblock[section_end_index[0]].split()[-1]
        
    return pd.DataFrame({'CRD Number':[CRD_number],
                         'Broker Name':[broker_name]})

In [9]:
def get_broker_name_CRD_Pool(file):
    try:
        fitz_doc = fitz.open(file)
        page1 = fitz_doc.loadPage(0)
        page1_textblock = page1.getTextBlocks()
        page1_textblock = [x[4] for x in page1_textblock]
        broker_name_candiate = page1_textblock[0].split('\n')

        if len(broker_name_candiate)==1:
            broker_name = page1_textblock[1]
        else:
            broker_name = broker_name_candiate[1]

        section_start_index = [i for i in range(len(page1_textblock)) if page1_textblock[i]=='Section Title']
        section_end_index = [i for i in range(len(page1_textblock)) if 'CRD# ' in page1_textblock[i]]

        if len(section_end_index) == 0:
            CRD_number = 'NO CRD Number'
        else:
            CRD_number = page1_textblock[section_end_index[0]].split()[-1]

        return (file,broker_name,CRD_number)
    except:
        print(file)
        
        return (file,'Error','Error')

1. /home/ubuntu/Broker/individual_355047.pdf - Failed to load pdf
2. 

In [10]:
def get_section_info(fitz_doc):
    page1 = fitz_doc.loadPage(0)
    page1_textblock = page1.getTextBlocks()
    page1_textblock = [x[4] for x in page1_textblock]
    
    section_start_index = [i for i in range(len(page1_textblock)) if page1_textblock[i]=='Section Title']
    section_end_index = [i for i in range(len(page1_textblock)) if 'CRD# ' in page1_textblock[i]]
    
    if len(section_end_index) == 0:
        CRD_number = 'NO CRD Number'
    else:
        CRD_number = page1_textblock[section_end_index[0]].split()[-1]
    
    if len(section_start_index)==0:
        return pd.DataFrame({'CRD Number':[CRD_number],
                             'Section Titles':['No Section'],
                             'Section Pages':['No Section']})
    section_titles = page1_textblock[(section_start_index[0]+1):section_end_index[0]]
    
    section_pages = page1_textblock[(section_end_index[0]+1):(section_end_index[0]+len(section_titles)+1)]
    
    return pd.DataFrame({'CRD Number':[CRD_number]*len(section_titles),
                         'Section Titles':section_titles,
                         'Section Pages':section_pages})

In [11]:
def get_section_info_Pool(file):
    CRD_number = re.findall(r'\d+',file)[0]
    try:
        fitz_doc = fitz.open(file)
        page1 = fitz_doc.loadPage(0)
        page1_textblock = page1.getTextBlocks()
        page1_textblock = [x[4] for x in page1_textblock]

        section_start_index = [i for i in range(len(page1_textblock)) if page1_textblock[i]=='Section Title']
        section_end_index = [i for i in range(len(page1_textblock)) if 'CRD# ' in page1_textblock[i]]

        if len(section_start_index)==0:
            return pd.DataFrame({'CRD Number':[CRD_number],
                                 'Section Titles':['No Section'],
                                 'Section Pages':['No Section']})
        section_titles = page1_textblock[(section_start_index[0]+1):section_end_index[0]]

        section_pages = page1_textblock[(section_end_index[0]+1):(section_end_index[0]+len(section_titles)+1)]

        return pd.DataFrame({'CRD Number':[CRD_number]*len(section_titles),
                             'Section Titles':section_titles,
                             'Section Pages':section_pages})
    except:
        print(file)
        return pd.DataFrame({'CRD Number':[CRD_number],
                             'Section Titles':['No Section'],
                             'Section Pages':['No Section']})

In [12]:
def get_branch_office(fitz_doc):
    page1 = fitz_doc.loadPage(0)
    page1_textblock = page1.getTextBlocks()
    page1_textblock = [x[4] for x in page1_textblock]
    
    section_start_index = [i for i in range(len(page1_textblock)) if page1_textblock[i]=='Section Title']
    section_end_index = [i for i in range(len(page1_textblock)) if 'CRD# ' in page1_textblock[i]]
    
    if len(section_end_index) == 0:
        CRD_number = 'NO CRD Number'
    else:
        CRD_number = page1_textblock[section_end_index[0]].split()[-1]
    
    page_summary = fitz_doc.loadPage(2)
    page_summary_textblock = page_summary.getTextBlocks()
    page_summary_textblock = [x[4] for x in page_summary_textblock]
    
    if (any(['This broker is not currently registered' in x for x in page_summary_textblock]) or
        any(['This broker is not currently registered with' in x and 'FINRA firm' in x for x in page_summary_textblock])):
        Firm_CRD = 'NOT FOUND'
        Start_Date = 'NOT FOUND'
        Company_Info = 'NOT FOUND'
    elif any(['Currently employed by and registered with' in x for x in page_summary_textblock]):
        cutting_index = [i for i,x in enumerate(page_summary_textblock) if 'Currently employed by and registered with' in x][0]
        Firm_CRD = re.findall(r'(\d+)',[x for x in page_summary_textblock[cutting_index+1].split('\n') if 'CRD' in x][0])[0]
        Start_Date = re.findall(r'(\d+/\d+/\d+)',page_summary_textblock[cutting_index+1].split('CRD#')[-1])
        if len(Start_Date)==0:
            Start_Date = 'NOT FOUND'
        else:
            Start_Date = Start_Date[0]
        Company_Info = page_summary_textblock[cutting_index+1].split('CRD#')[0].replace('\n',' ').strip()
    return pd.DataFrame({'CRD Number':[CRD_number],
                         'Company Info':[Company_Info],
                         'Company CRD':[Firm_CRD],
                         'Date Since':[Start_Date]})
    print(page_summary_textblock)
    

In [ ]:
section_info_df = pd.DataFrame()

for file in broker_pdf_list:
    print(file)
    doc = fitz.open(file)
    section_info_df = pd.concat([section_info_df,get_section_info(doc)])

In [ ]:
section_info_df.head(50)

In [13]:
def separate_consecutive_numbers(num_list):
    group_list = []
    for k, g in groupby([(x, i-x) for i,x in enumerate(num_list)], lambda x:x[1]):
        group_list.append(list(g))
    group_list = [[(c[0],i) for c in x] for i,x in enumerate(group_list)]
    return group_list

In [ ]:
file = 'C:/Users/i26184/Desktop/Broker/Broker_PDF/individual_2498722.pdf'

In [ ]:
doc = fitz.open(file)
section_info_df_temp =  get_section_info(doc)
CRD_number = re.findall(r'\d+',file)[0]

if section_info_df_temp['Section Titles'].iloc[0]!='No Section':
    broker_qualification_page = section_info_df_temp[section_info_df_temp['Section Titles']=='Broker Qualifications']['Section Pages'].iloc[0]
    pages = [int(x) for x in broker_qualification_page.split('-')]

    broker_page_textblock = []
    for page in range(min(pages),max(pages)+1):
        broker_page_textblock.extend(doc.loadPage(page+1).getTextBlocks())
else:
    broker_page_textblock = []
    for page in range(doc.pageCount):
        if any(['Broker Qualifications' in c[4] for c in doc.loadPage(page).getTextBlocks()]):
            broker_page_textblock.extend(doc.loadPage(page).getTextBlocks())

In [ ]:
len(broker_page_textblock)

In [ ]:
[x[4] for x in broker_page_textblock]

In [14]:
def extract_state(x):
    try:
        state_value = ['ALABAMA','ALASKA','ARIZONA','ARKANSAS','CALIFORNIA','COLORADO','CONNECTICUT','DELAWARE',
                   'FLORIDA','GEORGIA','HAWAII','IDAHO','ILLINOIS','INDIANA','IOWA','KANSAS','KENTUCKY','LOUISIANA',
                   'MAINE','MARYLAND','MASSACHUSETTS','MICHIGAN','MINNESOTA','MISSISSIPPI','MISSOURI','MONTANA',
                   'NEBRASKA','NEVADA','NEW HAMPSHIRE','NEW JERSEY','NEW MEXICO','NEW YORK','NORTH CAROLINA',
                   'NORTH DAKOTA','OHIO','OKLAHOMA','OREGON','PENNSYLVANIA','RHODE ISLAND','SOUTH CAROLINA','SOUTH DAKOTA',
                   'TENNESSEE','TEXAS','UTAH','VERMONT','VIRGINIA','WASHINGTON','WEST VIRGINIA','WISCONSIN','WYOMING',
                   'GUAM','PUERTO RICO','VIRGIN ISLANDS','DISTRICT OF COLUMBIA']
        state_condidates = [c for c in state_value if c.lower() in x.lower()]
        state_condidates_length = [len(c) for c in state_condidates]
        return ' '.join([y.capitalize() for y in [c for c in state_condidates if len(c)==max(state_condidates_length)][0].split()])
    except:
        return 'NOT FOUND'

In [75]:
def get_broker_qual_employ_from_pagetextblock(page_textblock):
    page_textblock = [x[4] for x in page_textblock]
    exam_section_start_index = [i for i,x in enumerate(page_textblock) if 'This individual has passed' in x or 'Industry Exams this Broker has Passed' in x][0]
    page_textblock = page_textblock[:exam_section_start_index]
    SRO_table_start_index = [i for i,x in enumerate(page_textblock) if all([c in x for c in ['SRO','Category','Date']])]
    US_Territory_table_start_index = [i for i,x in enumerate(page_textblock) if 'U.S. State/\nTerritory' in x]
    #page_block_dated = [(i,x) for i,x in enumerate(page_textblock) if len(re.sub(r'(\d+/\d+/\d+)','',x))!=len(x)]
    broker_qual_employ = pd.DataFrame()
    if len(SRO_table_start_index)!=0:
        
        if len(US_Territory_table_start_index)==0:
            SRO_table_end_index = SRO_table_start_index[1:]+[len(page_textblock)]
        else:
            SRO_table_end_index = SRO_table_start_index[1:] + [US_Territory_table_start_index[0]]
        SRO_grouped = []
        for i in range(len(SRO_table_start_index)):
            SRO_grouped.append(page_textblock[SRO_table_start_index[i]:SRO_table_end_index[i]])
        
        for SRO_grouped_each in SRO_grouped:
            SRO_block_dated = [(i,x) for i,x in enumerate(SRO_grouped_each) if len(re.sub(r'(\d+/\d+/\d+)','',x))!=len(x)]
            for i,x in SRO_block_dated:
                print(i,x)
                if len(re.sub(r'(\d+/\d+/\d+)','',SRO_grouped_each[i-1]))!=len(SRO_grouped_each[i-1]):
                    #print(x)
                    Date = re.findall(r'(\d+/\d+/\d+)',x)[0]
                    x = x.split(Date)[0]
                    SRO_or_US_Territory = x.split('\n')[0]
                    x = x.split(SRO_or_US_Territory)[-1]
                    #print(x)
                    Category = ' '.join([c for c in x.split() if c.capitalize()==c])
                    #print(Category)
                    Status = ' '.join([c for c in x.split(Category)[-1].split() if c.upper()==c])
                    broker_qual_employ_temp = pd.DataFrame({'Table':['SRO'],
                                                       'SRO or US Territory':[SRO_or_US_Territory],
                                                       'Category':[Category],
                                                       'Status':[Status],
                                                       'Date':[Date]})
                    broker_qual_employ = pd.concat([broker_qual_employ,broker_qual_employ_temp])
                else:
                    if '\n' in x:
                        #print('check')
                        Date = re.findall(r'(\d+/\d+/\d+)',x)[0]
                        x = x.split(Date)[0]
                        if all([c in SRO_grouped_each[i-1] for c in ['SRO','Category','Date']]):
                            SRO_or_US_Territory = x.split('\n')[0]
                        else:
                            SRO_or_US_Territory = SRO_grouped_each[i-1].replace('\n',' ')
                        #x = x.split(SRO_or_US_Territory)[-1]
                        #print(x)
                        Category = ' '.join([c for c in x.split() if c.capitalize()==c])
                        Status = ' '.join([c for c in x.split(Category)[-1].split() if c.upper()==c])
                        broker_qual_employ_temp = pd.DataFrame({'Table':['SRO'],
                                                           'SRO or US Territory':[SRO_or_US_Territory],
                                                           'Category':[Category],
                                                           'Status':[Status],
                                                           'Date':[Date]})
                        broker_qual_employ = pd.concat([broker_qual_employ,broker_qual_employ_temp])
            
    if len(US_Territory_table_start_index)!=0:        
        US_Territory_table_end_index = US_Territory_table_start_index[1:]+[len(page_textblock)]


    
        US_Territory_grouped = []


        for i in range(len(US_Territory_table_start_index)):
            US_Territory_grouped.append(page_textblock[US_Territory_table_start_index[i]:US_Territory_table_end_index[i]])


    
    

        for US_Territory_grouped_each in US_Territory_grouped:
            US_Territory_block_dated = [(i,x) for i,x in enumerate(US_Territory_grouped_each) if len(re.sub(r'(\d+/\d+/\d+)','',x))!=len(x)]

            for i,x in US_Territory_block_dated:
                #print(i,x)
                if extract_state(x)!='NOT FOUND':
                    #print(x)
                    Date = re.findall(r'(\d+/\d+/\d+)',x)[0]
                    x = x.split(Date)[0]
                    Status = x.split('\n')[-1].strip()
                    SRO_or_US_Territory = extract_state(x)
                    Category = x.split('\n')[0].split(SRO_or_US_Territory)[-1].strip()
                    broker_qual_employ_temp = pd.DataFrame({'Table':['US Territory'],
                                                            'SRO or US Territory':[SRO_or_US_Territory],
                                                            'Category':[Category],
                                                            'Status':[Status],
                                                            'Date':[Date]})
                    broker_qual_employ = pd.concat([broker_qual_employ,broker_qual_employ_temp])
                else:
                    #print(x)
                    Date = re.findall(r'(\d+/\d+/\d+)',x)[0]
                    x = x.split(Date)[0]
                    Status = x.split('\n')[-1].strip()
                    SRO_or_US_Territory = extract_state(US_Territory_grouped_each[i-1].replace('\n',' '))

                    if SRO_or_US_Territory == 'NOT FOUND':
                        continue
                    else:
                        Category = x.split('\n')[0].split(SRO_or_US_Territory)[-1].strip()
                        broker_qual_employ_temp = pd.DataFrame({'Table':['US Territory'],
                                                                'SRO or US Territory':[SRO_or_US_Territory],
                                                                'Category':[Category],
                                                                'Status':[Status],
                                                                'Date':[Date]})
                        broker_qual_employ = pd.concat([broker_qual_employ,broker_qual_employ_temp])
    return broker_qual_employ

In [76]:
def get_broker_qual_employ_from_pagetextblock_Pool(file):
    try:
        #print(file)
        doc = fitz.open(file)
        section_info_df_temp =  get_section_info(doc)
        CRD_number = re.findall(r'\d+',file.split('/')[-1])[0]
        #print(CRD_number)
        
        if section_info_df_temp['Section Titles'].iloc[0]!='No Section':
            broker_qualification_page = section_info_df_temp[section_info_df_temp['Section Titles']=='Broker Qualifications']['Section Pages'].iloc[0]
            pages = [int(x) for x in broker_qualification_page.split('-')]

            broker_page_textblock = []
            for page in range(min(pages),max(pages)+1):
                broker_page_textblock.extend(doc.loadPage(page+1).getTextBlocks())
        else:
            broker_page_textblock = []
            for page in range(doc.pageCount):
                if any(['Broker Qualifications' in c[4] for c in doc.loadPage(page).getTextBlocks()]):
                    broker_page_textblock.extend(doc.loadPage(page).getTextBlocks())
        #print('Check')
        broker_qual_employ_temp =  get_broker_qual_employ_from_pagetextblock(broker_page_textblock)
        broker_qual_employ_temp['CRD Number'] = [CRD_number]*broker_qual_employ_temp.shape[0]
        #print(broker_qual_employ_temp)
        return broker_qual_employ_temp
    except:
        print('Failed')

In [79]:
file = 'C:/Users/i26184/Desktop/Broker/Broker_PDF\\individual_704717.pdf'

In [80]:
get_broker_qual_employ_from_pagetextblock_Pool(file)

,CRD Number


In [81]:
doc = fitz.open(file)
section_info_df_temp =  get_section_info(doc)
CRD_number = re.findall(r'\d+',file.split('/')[-1])[0]
#print(CRD_number)

if section_info_df_temp['Section Titles'].iloc[0]!='No Section':
    broker_qualification_page = section_info_df_temp[section_info_df_temp['Section Titles']=='Broker Qualifications']['Section Pages'].iloc[0]
    pages = [int(x) for x in broker_qualification_page.split('-')]

    broker_page_textblock = []
    for page in range(min(pages),max(pages)+1):
        broker_page_textblock.extend(doc.loadPage(page+1).getTextBlocks())
else:
    broker_page_textblock = []
    for page in range(doc.pageCount):
        if any(['Broker Qualifications' in c[4] for c in doc.loadPage(page).getTextBlocks()]):
            broker_page_textblock.extend(doc.loadPage(page).getTextBlocks())
#print('Check')

In [74]:
[x[4] for x in broker_page_textblock]

['www.finra.org/brokercheck\nUser Guidance',
 'Broker Qualifications',
 'Registrations',
 'This section provides the self-regulatory organizations (SROs) and U.S. states/territories the broker is currently\nregistered and licensed with, the category of each license, and the date on which it became effective. This section also\nprovides, for every brokerage firm with which the broker is currently employed, the address of each branch where the\nbroker works.\nThis individual is currently registered with 1 SRO and is licensed in 0 U.S. states and territories through his or\nher employer.',
 'Employment 1 of 1\nFirm Name:',
 'Main Office Address:',
 'Firm CRD#:',
 'NYLIFE SECURITIES LLC',
 '5167',
 '51 MADISON AVE.\nROOM 713\nNEW YORK, NY  10010',
 'SRO Category Status Date',
 'FINRA\nInvest. Co and Variable Contracts APPROVED 07/20/1999',
 'Branch Office Locations',
 'NYLIFE SECURITIES LLC\n51 MADISON AVENUE\nNEW YORK, NY  10010',
 '2 ©2020 FINRA. All rights reserved. Report about YAN F. 

In [70]:
get_broker_qual_employ_from_pagetextblock(broker_page_textblock)

IndexError: list index out of range

In [ ]:
start_time = time.time()
p = Pool(8)
result = p.map(get_broker_qual_employ_from_pagetextblock_Pool,broker_pdf_list)
p.close()
p.join()
end_time = time.time()
print("--- %s seconds ---" % (end_time - start_time))

len(result)

In [ ]:
broker_qual_employ.shape

In [ ]:
broker_qual_employ

In [ ]:
broker_qual_employ['CRD Number'].unique()

In [ ]:
def get_broker_qual_employ_from_pagetextblock(page_textblock):
    page_textblock = [x[4] for x in page_textblock]
    
    page_block_dated = [(i,x) for i,x in enumerate(page_textblock) if len(re.sub(r'(\d+/\d+/\d+)','',x))!=len(x)]
    
    grouped_index_block_dated = separate_consecutive_numbers([x[0] for x in page_block_dated])
    
    #print(grouped_index_block_dated)
    broker_qual_employ = pd.DataFrame()
    for group_index in grouped_index_block_dated:
        index_temp = [x[0] for x in group_index]
        table_title = ' '.join(page_textblock[(min(index_temp)-2):min(index_temp)])
        #print(table_title)
        if all([c in table_title for c in ['SRO','Category','Date']]):
            #print('SRO')
            page_textblock_temp = [page_textblock[i] for i in index_temp]
            
            SRO = [x.split('\n')[0] for x in page_textblock_temp]
            Date = [re.findall(r'(\d+/\d+/\d+)',x)[0] for x in page_textblock_temp]
            
            page_textblock_temp = [re.sub(r'(\d+/\d+/\d+)','',x).split('\n')[1].strip() for x in page_textblock_temp]
            
            Status = [' '.join([c for c in x.split() if c.upper()==c]) for x in page_textblock_temp]
            Category = [' '.join([c for c in x.split() if c.capitalize()==c]) for x in page_textblock_temp]
            
            broker_qual_employ_temp = pd.DataFrame({'Table':'SRO',
                                               'SRO or US Territory':SRO,
                                               'Category':Category,
                                               'Status':Status,
                                               'Date':Date})
            
        elif all([c in table_title for c in ['U.S. State','Territory','Category','Date']]):
            #print('US')
            page_textblock_temp = [page_textblock[i] for i in index_temp]
            
            US_Territory = [' '.join(x.split('\n')[0].split()[:-1]) for x in page_textblock_temp]
            Date = [re.findall(r'(\d+/\d+/\d+)',x)[0] for x in page_textblock_temp]
            Category = [x.split('\n')[0].split()[-1] for x in page_textblock_temp]
            page_textblock_temp = [re.sub(r'(\d+/\d+/\d+)','',x).split('\n')[1].strip() for x in page_textblock_temp]
            
            Status = [' '.join([c for c in x.split() if c.upper()==c]) for x in page_textblock_temp]
            
            
            broker_qual_employ_temp = pd.DataFrame({'Table':'US Territory',
                                               'SRO or US Territory':US_Territory,
                                               'Category':Category,
                                               'Status':Status,
                                               'Date':Date})
        else:
            continue
        
        broker_qual_employ = pd.concat([broker_qual_employ,broker_qual_employ_temp])
    
    if broker_qual_employ.shape[0]==0:
        broker_qual_employ = pd.DataFrame({'Table':['NOT FOUND'],
                                            'SRO or US Territory':['NOT FOUND'],
                                            'Category':['NOT FOUND'],
                                            'Status':['NOT FOUND'],
                                            'Date':['NOT FOUND']})
    
    return broker_qual_employ

In [ ]:
def exam_class_identify_old(x):
    if all([c in x.lower() for c in ['principal','supervisory','exam']]):
        return 'Principal Supervisory Exam'
    elif all([c in x.lower() for c in ['general','industry','product','exam']]):
        return "General Industry Product Exam"
    else:
        return 'State Securities Law Exam'

In [ ]:
def exam_class_identify_new(each_exam_group):
    x = ' '.join(each_exam_group)
    if all([c in x.lower() for c in ['principal','supervisory','exam']]):
        return 'Principal Supervisory Exam'
    elif all([c in x.lower() for c in ['general','industry','product','exam']]):
        return "General Industry Product Exam"
    else:
        return 'State Securities Law Exam'

In [ ]:
def exam_name_identify_new(each_exam_group,i):
    if 'Examination' in each_exam_group[i]:
        return each_exam_group[i].split('Examination')[0] + ' Examination'
    else:
        x = ' '.join([each_exam_group[i-1], each_exam_group[i]])
        return x.split('Examination')[0] + ' Examination'

In [ ]:
def get_broker_qual_exam_from_pagetextblock(page_textblock):
    page_textblock = [x[4] for x in page_textblock]
    
    exam_section_index = [i for i,x in enumerate(page_textblock) if all([c in x.lower() for c in ['this individual has passed']])]
    
    broker_qual_exam = pd.DataFrame()
    
    if len(exam_section_index)!=0:
        exam_count = re.findall(r'[0-9]',page_textblock[exam_section_index[0]])
        
        principal_supervisory_exam = exam_count[0]
        general_industry_product_exam = exam_count[1]
        state_securities_law_exam = exam_count[2]
        
        page_textblock = page_textblock[exam_section_index[0]:]

        exam_index = [i for i,x in enumerate(page_textblock) if all([c in x for c in ['Exam','Category','Date']])]

        exam_group = []
        
        for i,j in zip(exam_index,exam_index[1:]+[len(page_textblock)]):
            exam_group.append(page_textblock[i:j])
        #print(exam_group)
        
        
        
        for each_exam_group in exam_group:
            index_dated = [(i,x) for i,x in enumerate(each_exam_group) if len(re.sub(r'(\d+/\d+/\d+)','',x))!=len(x)]
            #ach_exam_group_dated = each_exam_group[index_dated]
            if len(index_dated)==0:
                
                continue
            elif any([True if all([c in x for c in ['Class','Exam','Category','Date']]) else False for x in each_exam_group]):
                Date = [re.findall(r'(\d+/\d+/\d+)',each_exam_group[i])[0] for i,text in index_dated]
                Exam_Class = [exam_class_identify_old(each_exam_group[i]) for i,text in index_dated] 
                Exam_Name = [each_exam_group[i].split('Exams')[-1].split('Examination')[0]+' Examination' for i,text in index_dated]    
                Category = [re.sub(r'(\d+/\d+/\d+)','',each_exam_group[i].split('Examination')[-1].strip()).strip() for i,text in index_dated]
            else:
                Date = [re.findall(r'(\d+/\d+/\d+)',each_exam_group[i])[0] for i,text in index_dated]
                Exam_Class = [exam_class_identify_new(each_exam_group) for i,text in index_dated] 
                Exam_Name = [exam_name_identify_new(each_exam_group,i) for i,text in index_dated]    
                Category = [re.sub(r'(\d+/\d+/\d+)','CUTTING_POINT',each_exam_group[i]).split('CUTTING_POINT')[-1].strip() for i,text in index_dated]
            
            broker_qual_exam_temp = pd.DataFrame({'Exam Class':Exam_Class,
                                                  'Exam Name':Exam_Name,
                                                  'Category':Category,
                                                  'Date':Date})
            
            broker_qual_exam = pd.concat([broker_qual_exam,broker_qual_exam_temp])
        
    else:
        broker_qual_exam = pd.DataFrame()
        
    if broker_qual_exam.shape[0]==0:
        Date = ['NOT FOUND']
        Exam_Class = ['NOT FOUND']
        Exam_Name = ['NOT FOUND']
        Category = ['NOT FOUND']
        broker_qual_exam = pd.DataFrame({'Exam Class':Exam_Class,
                                                  'Exam Name':Exam_Name,
                                                  'Category':Category,
                                                  'Date':Date})
    return broker_qual_exam

In [ ]:
def get_regist_hist_from_pagetextblock(page_textblock):
    page_textblock = [x[4] for x in page_textblock]
    
    regist_hist_section_index = [i for i,x in enumerate(page_textblock) if all([c in x.lower() for c in ['Registration History']])]
    employ_hist_section_index = [i for i,x in enumerate(page_textblock) if all([c in x.lower() for c in ['Employment History']])]
    
    if len(regist_hist_section_index)!=0:
        if len(employ_hist_section_index)==0:
            employ_hist_section_index = [len(page_textblock)]
    
        page_textblock = page_textblock[regist_hist_section_index[0]:employ_hist_section_index[0]]
        
        index_dated = [(i,x) for i,x in enumerate(page_textblock) if len(re.sub(r'(\d+/\d+)','',x))!=len(x)]
        
        Start_Date = [re.findall(r'(\d+/\d+)',x)[0] for i,x in index_dated]
        End_Date = [re.findall(r'(\d+/\d+)',x)[1] for i,x in index_dated]
        
    else:
        registration_history = pd.DataFrame()
        
    return registration_history

In [ ]:
# 816782

In [ ]:
broker_name_df = pd.DataFrame()
for file in broker_pdf_list:
    #print(file)
    doc = fitz.open(file)
    broker_name_df = pd.concat([broker_name_df,get_broker_name_CRD(doc)])

In [ ]:
broker_name_df.tail(50)

In [ ]:
def get_broker_qual_exam_from_pagetextblock_Pool(file):
    try:
        #print(file)
        doc = fitz.open(file)
        section_info_df_temp =  get_section_info(doc)
        CRD_number = re.findall(r'\d+',file)[0]
        
        if section_info_df_temp['Section Titles'].iloc[0]!='No Section':
            broker_qualification_page = section_info_df_temp[section_info_df_temp['Section Titles']=='Broker Qualifications']['Section Pages'].iloc[0]
            pages = [int(x) for x in broker_qualification_page.split('-')]

            broker_page_textblock = []
            for page in range(min(pages),max(pages)+1):
                broker_page_textblock.extend(doc.loadPage(page+1).getTextBlocks())
        else:
            broker_page_textblock = []
            for page in range(doc.pageCount):
                if any(['Broker Qualifications' in c[4] for c in doc.loadPage(page).getTextBlocks()]):
                    broker_page_textblock.extend(doc.loadPage(page).getTextBlocks())
        #print('Check')
        broker_qual_exam_temp =  get_broker_qual_exam_from_pagetextblock(broker_page_textblock)
        broker_qual_exam_temp['CRD Number'] = [CRD_number]*broker_qual_exam_temp.shape[0]
        #print(broker_qual_employ_temp)
        return broker_qual_exam_temp
    except:
        print(file)


In [ ]:
start_time = time.time()
p = Pool(8)
result = p.map(get_broker_qual_exam_from_pagetextblock_Pool,broker_pdf_list)
p.close()
p.join()
end_time = time.time()
print("--- %s seconds ---" % (end_time - start_time))

len(result)

In [ ]:
result_df = pd.concat(result)

In [ ]:
result_df.shape

In [ ]:
result_df

In [ ]:
result_df.to_csv('broker_exam.csv',index = False)

In [ ]:
def get_broker_qual_employ_from_pagetextblock_Pool(file):
    try:
        #print(file)
        doc = fitz.open(file)
        section_info_df_temp =  get_section_info(doc)
        CRD_number = re.findall(r'\d+',file)[0]
        
        if section_info_df_temp['Section Titles'].iloc[0]!='No Section':
            broker_qualification_page = section_info_df_temp[section_info_df_temp['Section Titles']=='Broker Qualifications']['Section Pages'].iloc[0]
            pages = [int(x) for x in broker_qualification_page.split('-')]

            broker_page_textblock = []
            for page in range(min(pages),max(pages)+1):
                broker_page_textblock.extend(doc.loadPage(page+1).getTextBlocks())
        else:
            broker_page_textblock = []
            for page in range(doc.pageCount):
                if any(['Broker Qualifications' in c[4] for c in doc.loadPage(page).getTextBlocks()]):
                    broker_page_textblock.extend(doc.loadPage(page).getTextBlocks())
        #print('Check')
        broker_qual_employ_temp =  get_broker_qual_employ_from_pagetextblock(broker_page_textblock)
        broker_qual_employ_temp['CRD Number'] = [CRD_number]*broker_qual_employ_temp.shape[0]
        #print(broker_qual_employ_temp)
        return broker_qual_employ_temp
    except:
        print(file)


In [ ]:
get_broker_qual_employ_from_pagetextblock_Pool('/home/ubuntu/Broker/individual_2498722.pdf')

In [ ]:
file = '/home/ubuntu/Broker/individual_2498722.pdf'

In [ ]:
doc = fitz.open(file)
section_info_df_temp =  get_section_info(doc)
CRD_number = re.findall(r'\d+',file)[0]

In [ ]:
if section_info_df_temp['Section Titles'].iloc[0]!='No Section':
    broker_qualification_page = section_info_df_temp[section_info_df_temp['Section Titles']=='Broker Qualifications']['Section Pages'].iloc[0]
    pages = [int(x) for x in broker_qualification_page.split('-')]

    broker_page_textblock = []
    for page in range(min(pages),max(pages)+1):
        broker_page_textblock.extend(doc.loadPage(page+1).getTextBlocks())
else:
    broker_page_textblock = []
    for page in range(doc.pageCount):
        if any(['Broker Qualifications' in c[4] for c in doc.loadPage(page).getTextBlocks()]):
            broker_page_textblock.extend(doc.loadPage(page).getTextBlocks())

In [ ]:
broker_page_textblock

In [ ]:
get_broker_qual_employ_from_pagetextblock(broker_page_textblock)

In [ ]:
page_textblock = broker_page_textblock

In [ ]:
page_textblock = [x[4] for x in page_textblock]
    
page_block_dated = [(i,x) for i,x in enumerate(page_textblock) if len(re.sub(r'(\d+/\d+/\d+)','',x))!=len(x)]

grouped_index_block_dated = separate_consecutive_numbers([x[0] for x in page_block_dated])

In [ ]:
grouped_index_block_dated[0]

In [ ]:
broker_qual_employ = pd.DataFrame()
for group_index in grouped_index_block_dated:
    print(group_index)
    index_temp = [x[0] for x in group_index]
    table_title = ' '.join(page_textblock[(min(index_temp)-2):min(index_temp)])
    print(table_title)
    if all([c in table_title for c in ['SRO','Category','Date']]):
        print('SRO')
        page_textblock_temp = [page_textblock[i] for i in index_temp]

        SRO = [x.split('\n')[0] for x in page_textblock_temp]
        Date = [re.findall(r'(\d+/\d+/\d+)',x)[0] for x in page_textblock_temp]

        page_textblock_temp = [re.sub(r'(\d+/\d+/\d+)','',x).split('\n')[1].strip() for x in page_textblock_temp]

        Status = [' '.join([c for c in x.split() if c.upper()==c]) for x in page_textblock_temp]
        Category = [' '.join([c for c in x.split() if c.capitalize()==c]) for x in page_textblock_temp]

        broker_qual_employ_temp = pd.DataFrame({'Table':'SRO',
                                           'SRO or US Territory':SRO,
                                           'Category':Category,
                                           'Status':Status,
                                           'Date':Date})

    elif all([c in table_title for c in ['U.S. State','Territory','Category','Date']]):
        print('US')
        page_textblock_temp = [page_textblock[i] for i in index_temp]

        US_Territory = [' '.join(x.split('\n')[0].split()[:-1]) for x in page_textblock_temp]
        Date = [re.findall(r'(\d+/\d+/\d+)',x)[0] for x in page_textblock_temp]
        Category = [x.split('\n')[0].split()[-1] for x in page_textblock_temp]
        page_textblock_temp = [re.sub(r'(\d+/\d+/\d+)','',x).split('\n')[1].strip() for x in page_textblock_temp]

        Status = [' '.join([c for c in x.split() if c.upper()==c]) for x in page_textblock_temp]


        broker_qual_employ_temp = pd.DataFrame({'Table':'US Territory',
                                           'SRO or US Territory':US_Territory,
                                           'Category':Category,
                                           'Status':Status,
                                           'Date':Date})

In [ ]:
def get_broker_qual_employ_from_pagetextblock(page_textblock):
    page_textblock = [x[4] for x in page_textblock]
    
    page_block_dated = [(i,x) for i,x in enumerate(page_textblock) if len(re.sub(r'(\d+/\d+/\d+)','',x))!=len(x)]
    
    grouped_index_block_dated = separate_consecutive_numbers([x[0] for x in page_block_dated])
    
    #print(grouped_index_block_dated)
    broker_qual_employ = pd.DataFrame()
    for group_index in grouped_index_block_dated:
        index_temp = [x[0] for x in group_index]
        table_title = ' '.join(page_textblock[(min(index_temp)-2):min(index_temp)])
        #print(table_title)
        if all([c in table_title for c in ['SRO','Category','Date']]):
            #print('SRO')
            page_textblock_temp = [page_textblock[i] for i in index_temp]
            
            SRO = [x.split('\n')[0] for x in page_textblock_temp]
            Date = [re.findall(r'(\d+/\d+/\d+)',x)[0] for x in page_textblock_temp]
            
            page_textblock_temp = [re.sub(r'(\d+/\d+/\d+)','',x).split('\n')[1].strip() for x in page_textblock_temp]
            
            Status = [' '.join([c for c in x.split() if c.upper()==c]) for x in page_textblock_temp]
            Category = [' '.join([c for c in x.split() if c.capitalize()==c]) for x in page_textblock_temp]
            
            broker_qual_employ_temp = pd.DataFrame({'Table':'SRO',
                                               'SRO or US Territory':SRO,
                                               'Category':Category,
                                               'Status':Status,
                                               'Date':Date})
            
        elif all([c in table_title for c in ['U.S. State','Territory','Category','Date']]):
            #print('US')
            page_textblock_temp = [page_textblock[i] for i in index_temp]
            
            US_Territory = [' '.join(x.split('\n')[0].split()[:-1]) for x in page_textblock_temp]
            Date = [re.findall(r'(\d+/\d+/\d+)',x)[0] for x in page_textblock_temp]
            Category = [x.split('\n')[0].split()[-1] for x in page_textblock_temp]
            page_textblock_temp = [re.sub(r'(\d+/\d+/\d+)','',x).split('\n')[1].strip() for x in page_textblock_temp]
            
            Status = [' '.join([c for c in x.split() if c.upper()==c]) for x in page_textblock_temp]
            
            
            broker_qual_employ_temp = pd.DataFrame({'Table':'US Territory',
                                               'SRO or US Territory':US_Territory,
                                               'Category':Category,
                                               'Status':Status,
                                               'Date':Date})
        else:
            continue
        
        broker_qual_employ = pd.concat([broker_qual_employ,broker_qual_employ_temp])
    
    if broker_qual_employ.shape[0]==0:
        broker_qual_employ = pd.DataFrame({'Table':['NOT FOUND'],
                                            'SRO or US Territory':['NOT FOUND'],
                                            'Category':['NOT FOUND'],
                                            'Status':['NOT FOUND'],
                                            'Date':['NOT FOUND']})
    
    return broker_qual_employ

In [ ]:
#print(file)
        doc = fitz.open(file)
        section_info_df_temp =  get_section_info(doc)
        CRD_number = re.findall(r'\d+',file)[0]
        
        if section_info_df_temp['Section Titles'].iloc[0]!='No Section':
            broker_qualification_page = section_info_df_temp[section_info_df_temp['Section Titles']=='Broker Qualifications']['Section Pages'].iloc[0]
            pages = [int(x) for x in broker_qualification_page.split('-')]

            broker_page_textblock = []
            for page in range(min(pages),max(pages)+1):
                broker_page_textblock.extend(doc.loadPage(page+1).getTextBlocks())
        else:
            broker_page_textblock = []
            for page in range(doc.pageCount):
                if any(['Broker Qualifications' in c[4] for c in doc.loadPage(page).getTextBlocks()]):
                    broker_page_textblock.extend(doc.loadPage(page).getTextBlocks())
        #print('Check')
        broker_qual_employ_temp =  get_broker_qual_employ_from_pagetextblock(broker_page_textblock)

In [ ]:
start_time = time.time()
p = Pool(8)
result = p.map(get_broker_qual_employ_from_pagetextblock_Pool,broker_pdf_list)
p.close()
p.join()
end_time = time.time()
print("--- %s seconds ---" % (end_time - start_time))

len(result)

In [ ]:
result_df = pd.concat(result)

In [ ]:
result_df.shape

In [ ]:
result_df

In [ ]:
result_df.to_csv('broker_registration.csv',index = False)

In [ ]:
broker_qual_employ = pd.DataFrame()
broker_qual_exam = pd.DataFrame()
broker_name_CRD = pd.DataFrame()
broker_branch_office = pd.DataFrame()
for file in broker_pdf_list:
    print(file)
    doc = fitz.open(file)
    section_info_df_temp =  get_section_info(doc)
    #print(section_info_df_temp)
    CRD_number = section_info_df_temp.loc[0,'CRD Number']
        
    if section_info_df_temp['Section Titles'].iloc[0]!='No Section':
        broker_qualification_page = section_info_df_temp[section_info_df_temp['Section Titles']=='Broker Qualifications']['Section Pages'].iloc[0]
        pages = [int(x) for x in broker_qualification_page.split('-')]

        borker_page_textblock = []
        for page in range(min(pages),max(pages)+1):
            borker_page_textblock.extend(doc.loadPage(page+1).getTextBlocks())
    else:
        borker_page_textblock = []
        for page in range(doc.pageCount):
            if any(['Broker Qualifications' in c[4] for c in doc.loadPage(page).getTextBlocks()]):
                borker_page_textblock.extend(doc.loadPage(page).getTextBlocks())
            
    broker_qual_employ_temp =  get_broker_qual_employ_from_pagetextblock(borker_page_textblock)
    broker_qual_exam_temp =  get_broker_qual_exam_from_pagetextblock(borker_page_textblock)
    broker_qual_employ_temp['CRD Number'] = [CRD_number]*broker_qual_employ_temp.shape[0]
    broker_qual_exam_temp['CRD_Number'] = [CRD_number]*broker_qual_exam_temp.shape[0]
    
    broker_qual_employ = pd.concat([broker_qual_employ,broker_qual_employ_temp])
    broker_qual_exam = pd.concat([broker_qual_exam,broker_qual_exam_temp])
    broker_name_CRD = pd.concat([broker_name_CRD,get_broker_name_CRD(doc)])
    broker_branch_office = pd.concat([broker_branch_office,get_branch_office(doc)])

In [ ]:
broker_qual_exam.to_csv('C:\\Users\\i26184\\Desktop\\Broker\\broker_qual_exam.csv',index= False)

In [ ]:
broker_qual_employ.to_csv('C:\\Users\\i26184\\Desktop\\Broker\\broker_qual_employ.csv',index= False)

In [ ]:
broker_name_CRD.to_csv('C:\\Users\\i26184\\Desktop\\Broker\\broker_name_CRD.csv',index= False)

In [ ]:
broker_branch_office.to_csv('C:\\Users\\i26184\\Desktop\\Broker\\broker_branch_office.csv',index= False)

In [ ]:
broker_branch_office[broker_branch_office['Company Info']!='NOT FOUND'].shape

In [ ]:
broker_qual_employ['Category'].unique()

In [ ]:
doc = fitz.open('/home/ubuntu/Broker/individual_816727.pdf')

In [ ]:
page1 = doc.loadPage(6).getTextBlocks()

In [ ]:
page1 = [x[4] for x in page1]

In [ ]:
page1

In [ ]:
get_SRO_registration_from_pagetextblock(doc.loadPage(3).getTextBlocks())

In [ ]:
page1_textblock = page1.getTextBlocks()

In [ ]:
page1_textblock